# Empleos_IT

El proposito de este proyecto verificar l factibilidad de hacer scraping en este sitio

- https://empleosti.com.mx/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt-get update
!pip install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
from datetime import date
from time import sleep
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import pandas as pd
import requests
from urllib import request

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('disable-infobars')
chrome_options.add_argument("window-size=1200x600")
chrome_options.add_argument("--disable-gpu")

In [8]:
#Diccionario de grados academicos
diccionario_academic_degree = {
    #Espanol
    #La ingenieria tambien es una licenciatura
    r"(licenciatura|ingenier(i|í)a|licenciad(o|a)\s(en|de)|ingenier(o|a)\s(en|de)|\sLic\.\s|\sIng\.\s)" : "Licenciatura", 
    r"maestr(i|í)a" : "Maestria", 
    r"doctorado" : "Doctorado", 
    #English
    r"(bachelor('s)?|\sB(\.)?S(\.)?\sin)" : "Bachelor", 
    r"(master('s)?|\sM(\.)?S(\.)?(\s|\/P(H|h)(\.)?D)(or|in))" : "Master", 
    r"\bph(\.)?d\b" : "PhD"
}

#Diccionario de Soft Skills
#Creo que hay pocos soft skill en español ya que la mayoria de las descripcion estaban en ingles
diccionario_soft_skill ={
    #Spanish
    r"\banal(í|i)tica\b" : "Capacidad Analitica", 
    r"\b(creatividad|creativ(a|o)s?)\b" : "Creatividad",
    r"\b(empuje\spor\sobtener\sresultados\b|enfocado\sa\sresultados)\b" : "Enfocado a resultados",
    r"\bmanejo\sde\sconflictos\b" : "Manejo de conflictos",
    r"\binspirar\b" :"Inspirar",
    r"\b(organizado|organizas)\b" : "Organizado",
    r"\bproactiv(a|o)" : "Proactivo",
    r"\bresoluci(ó|o)n\sde\sproblemas\b" : "Resolucion de problemas",
    r"\btrabajo\sen\sequipo\b" : "Trabajo en equipo",
    r"\btrabajar?\sde\sforma\sindependiente\b" : "Trabaja de forma independiente", 
    #English
    r"\b(analytical|analytics?)\b" : "Analitical", 
    r"\battention\sto\sdetails?\b" : "Attention to details", 
    r"\bcoach\b" : "Coach", 
    r"\bcollaborat(e|es|ing|ion|ively)\b" : "Collaborate", 
    r"\bcommunicat(e|es|ion|ing)\b" : "Comunication skills",
    r"\b(continuously\slearn|learning|learn)\b" : "Continuously learn", 
    r"\bcreativity\b" : "Creativity", 
    r"\bcuriosity\b" : "Curiosity",
    r"\bcritical\sthinking\b" : "Critical Thinking", 
    r"\bdetail(\-|\s)oriented\b" : "Detail Oriented",
    r"\bdealing\s(well)?\swith\sambiguity\b" : "Dealing well with ambiguity",
    r"\bdynamic\senvironment\b":"Dynamic enviroment",
    r"\befficiency\b" : "Efficiency",
    r"\benthusiasm\b" : "Enthusiasm", 
    r"\bgroup\sintegration\b" : "Group integration",
    r"\bgrowth\smindset\b" : "Growth Mindset",
    r"\b(H|h)igh\sstandars\b" : "High Standars",
    r"\b(independent\sworker|works?\sindependently)\b" : "Independent Worker",
    r"\b(innovati(on|ve)|innovators)\b" : "Innovation",
    r"\binvolved\b" : "Involved",
    r"\binterpersonal\b" : "Interpersonal", 
    r"\b(leadership|lead\sa\steam)\b" : "Leadership",
    r"\bmanaging\s\w*\steams\b" : "Managing teams",
    r"\bmentor\b" : "Mentor",
    r"\bmotivated\b" : "Motivated",
    r"\bmultifunctional\stasking\b" : "Multifunctional tasking",
    r"\borganization(al)?\b" : "Organization",
    r"\bplanification\b" : "Planification",
    r"\bpositive\sattitude\b" : "Positive attitude",
    r"\bprioritizing\sneeds\b" : "Prioritizing needs",
    r"\bproactivity\b" : "Proactivity",
    r"\b(problem(\-|\s)(solving|solver))\b" : "Problem solving",
    r"\bself(\s|\-)directed\b" : "Self directed", 
    r"\bself(\s|\-)management\b" : "Self Management", 
    r"\bself(\s|\-)starter\b" : "Self Starter", 
    r"\bstrategic\b" : "strategic",
    r"\btarget(\s)?[-](\s)?based\sresults\b" : "Target – Based results",
    r"\bteam\splayer" : "Team Player",
    r"\bteam(\s)?work" : "Team Work", 
    r"\b(written|verbal|oral)(\sand\s|\/|\s[&]\s)(written|verbal|oral)\b" : "Written and oral communication",
    r"\b(cross(\-|\s)functional\s(teams|partners)|interdisciplinary|multiple\sfunctional\sgroups)\b" : "Working with cross functional teams"
}

#Diccionario de idiomas
diccionario_languages = {    
    #Ingles
    r"\b(advanced?\s(or\sfluent\s)?(in\s)?english|C1)\b" : "Advanced English",   
    r"\bintermediate(\s?\-\s?advanced?)?\senglish\b" : "Intermediate English",
    r"\b(F|f)luent\sin\s(w+\sand\s)?english\b" : "Fluent in English",
    r"\b(biling(ü|u)e|bi(\s|\-)lingual)\b" : "Bilingue",
    #Español
    r"\bingl(e|é|é)s[:]?\s(intermedio[-])?avanzado\b" : "Ingles Avanzado",
    r"\bingl(e|é|é)s\st(e|é)cnico\b" : "Ingles Tecnico",
    r"\bingl(e|é|é)s[:]?\s((N|n)ivel\s)?intermedio\b" : "Ingles Intermedio",
    r"\bingl(e|é|é)s[:]?\s((N|n)ivel\sintermedio\s)?alto\b" : "Ingles Avanzado",
    #Algunas descripciones no especificaban nivel, solo mencionaban que se necesitaba ingles
    r"\bingl(e|é|é)s\b" : "Ingles",
    r"\benglish\b" : "English",
}

#Patron de years of experience 
busqueda_years_experience = r"((\+?([0-4]?[0-9]|[1-9])\+?(\s?[-]\s?|\sto\s))?\+?([0-4]?[0-9]|[1-9])\+?\s((Y|y)ears?|a(ñ|ñ)os?))"
patron_year = re.compile(busqueda_years_experience)


In [1]:
#Diccionario de Hard skills
diccionario_HardSkill  = {
      #Lenguajes
    r"\b(c|\\/c\\+\\+|c\\+\\+)\b" : "C/C++",
    r"\b(c#\\/\\.net|c#|\\.net)\b" : "C#/.Net",
    r"\bobjective(\\-|\s)c\b" : "Objective-C",
    r"\bpython(\s[2-3](\\.[0-9])?)?\b" : "Python",
    r"\bgroovy\b" : "Groovy",
    r"\bhtml(5)?\b" : "HTML",
    r"\bCSS\b" : "CSS",
    r"\bjava\b" : "Java",
    r"\bjavascript\b" : "JavaScript",
    r"\btypescript\b" : "TypeScript",
    r"\br\b" : "R",
    r"\bt(\s|\\-)sql\b" : "T-SQL",
    r"\bsql\b" : "SQL",
    r"\bnosql\b" : "NoSQL",
    r"\bruby\b" : "Ruby",
    r"\bscala\b" : "Scala",
    r"\bclojure\b" : "Clojure",
    r"\belixir\b" : "Elixir",
    r"\berlang\b" : "Erlang",
    r"\bgolang\b" : "Golang",
    r"\bhaskell\b" : "Haskell",
    r"\bkotlin\b" : "Kotlin",
    r"\bperl\b" : "Perl",
    r"\bphp\b" : "PHP",
    r"\bswift\b" : "Swift",
    r"\bbash\b" : "Bash",
    #libraries
    r"\bopen(\s)?cv\b" : "OpenCV",
    r"\bselenium\b" : "Selenium", 
        #js
    r"\bbackbone(\\.js)?\b" : "Backbone.js",
    r"\bember(\\.js)?\b" : "Ember.js",
    r"\bjquery\b" : "JQuery",
    r"\bredux\b" : "Redux",
    r"\bd3(\\.js)?\b" : "D3.js",
    r"\sknockout((\\.)?js)" : "KnockoutJS", 
        #javascript testing
    r"\bjest\b" : "Jest",
    r"\bpuppeteer\b" : "Puppeteer",
    r"\bcupress\b" : "Cypress",
        #javascript Task runners
    r"\bnpm\sscripts\b" : "NPM Scripts",
    r"\bgulp\b" : "Gulp",
    r"\bgrunt\b" : "Grunt", 
        #css
    r"\bflexbox\b" : "Flexbox",
    r"\bcss\sgrid\b" : "CSS grid",
        #css pre processors
    r"\bsass\b" : "SASS",
    r"\bpostcss\b" : "PostCSS",
        #css framework
    r"\bstyled\scomponents\sreact\b" : "Styled Components React",
    r"\bbootstraps\b" : "Bootstraps",
        #python
    r"\bpytest\b" : "Pytest",
    r"\bmatplotlib\b" : "Matplotlib",
    r"\bcaret\b": "Caret", 
    r"\bkeras\b" : "Keras",
    r"\bnumpy\b" : "NumPy",   
    r"\bpandas\b" : "Pandas",
    r"\bpytorch\b" : "PyTorch",
    r"\bseaborn\b" : "Seaborn",
    r"\bscipy\b" : "SciPy",
    r"\bpysark\b" : "PySpark", 
    r"\bspacy\b" : "spaCy",
    r"\bscikit(\\-|\s)?learn\b" : "Scikit-Learn",
    r"tensor(\s)?flow" : "TensorFlow", 
    r"\btheano\b" : "Theano",
        #R
    r"\bshiny\b" : "Shiny", 
    r"\bmlr\b" : "mlr",
    #DataBase
    r"\bcassandra\b" : "Cassandra",
    r"\bmongo(db)?\b" : "MongoDB",
    r"\bmysql\b" : "MySQL",
    r"\bpostgres(ql)?\b" : "PostgreSQL", 
    r"\bmaria(\sdb)\b" : "MariaDB",
    r"\b(sql(\s)?server|mssql)\b" : "SQL server",
    r"\boracle\b" : "Oracle",
    r"\bdynamodb\b" : "DynamoDB",
    r"\binmemory\b" : "Inmemory", 
    r"\bredis\b" : "Redis", 
    #
    r"\bdata\s(warehouse|warehousing)\b" : "Data Warehouse",
    r"\bdata\slake\b" : "Data Lake",
    r"\bdatabricks\b" :  "DataBricks",
    #framework
    r"\bangular(js|\s10)?\b" : "Angular",
    r"\bdjango\b" : "django",
    r"\bflask\b" : "Flask",
    r"\blaravel\b" : "Laravel",
    r"\bnode(\\.js)?\b" : "Node.js",
    r"\breact\b" : "React",
    r"\bspring\b" : "Spring",
    r"\bvue(\\.js)?\b" : "Vue.js",
    r"\bnuxt(\\.js)?\b" : "Nuxt.js",
    r"\bnext(\\.js)?\b" : "Next.js",
    r"\bsvelte\b" : "Svelte",    
    #
    r"\banaconda\b" : "Anaconda", 
    r"jupyter(\snotebooks)?" : "Jupyter notebooks",
    r"\bstata\b" : "Stata",
    r"\scaffe(\s)?2\b" : "Caffe 2",
    r"matlab" : "MATLAB",
    r"\sSPSS(\sstatistics)?" : "SPSS",
    r"\b(microsoft)?\soffice\b" : "Office",  
    r"\bexcel\b" : "Excel",
    #Administracion 
    r"\bscrum\b" : "Scrum", 
    r"\bslack\b": "Slack", 
    r"\btrello\b" : "Trello",
    r"\bmiro\b" : "Miro",
    r"\bkanban(\s)?track\b" : "KanbanTrack",
    r"\bjira\b" : "Jira", 
    #movil
    r"\bandroid\b" : "Android",
    r"\biosb" : "IOS",
    #
    r"\blinux\b" : "Linux",
    r"\bunix\b" : "unix",
    #automatizacion
    r"\bansible\b" : "Ansible",
    r"\bdocker\b" : "Docker",
    r"\bjenkins\b" : "Jenkins", 
    r"\bkubernetes\b" : "Kubernetes",
    r"\bdigital\socean\b" : "Digital Ocean",
    r"\bheroku\b" : "HerokU",
    r"\bnetlify\b" : "Netlify",
    r"\bargo\b" : "Argo",
    r"\bdevops\b" : "Devops",
    #CI/CD
    r"\bgithub\sactions\b" : "Github Actions",
    r"\bjenikins(\sx)?\b" : "Jenikins",
    r"\bcircle\sci\b" : "Circle CI",
    #Deployment
    r"\bsurge\\.sn\b" : "Surge.sn",
    r"\bvercel\b" : "Vercel",
    r"\bfirebase\b" : "Firebase",
    r"\bnetlify\b" : "Netlify", 
    #apache
    r"\b(apache\s)?spark\b" : "Spark", 
    r"\b(apache\s)?hadoop\b" : "Hadoop", 
    r"\b(apache\s)?kafka\b" : "Kafka", 
    r"\b(apache\s)?flink\b" : "Flink",
    r"\b(apache\s)?druid\b" : "Druid",
    #aws
    r"\baws\b" : "AWS",
    r"\b((aws|amazon)(\s)?)?kinesis\b" : "Kinesis",
    r"\b((aws|amazon)(\s)?)?lambda\b" : "Lambda",
    r"\b((aws|amazon)(\s)?)?redis\b" : "Redis",
    r"\b((aws|amazon)(\s)?)?redshift\b" : "Redshift",
    r"\b((aws|amazon)(\s)?)?ec2\b" : "EC2",
    r"\b((aws|amazon)(\s)?)?dms\b" : "DMS",
    r"\b((aws|amazon)(\s)?)?glue\b" : "Glue",
    r"\b((aws|amazon)(\s)?)?athenea\b" : "Athenea",
    r"\b((aws|amazon)(\s)?)?sagemaker\b" : "Sagemaker",
    r"\b((aws|amazon)(\s)?)?s3\b" : "S3",
    r"\b((aws|amazon)(\s)?)?quicksight\b" : "Quicksight",
    r"\bblazing(\s)?db\b" : "BlazingDB",
    #azure
    r"\bazure\b" : "Azure",
    r"\b(azure\s)?cosmos(\s)?db\b" : "Cosmos DB",
    r"\b(azure\s)?blob\sstorage\b" : "Azure Blob Storage",
    #google
    r"\b(gcp|google\scloud\splatform)\b" : "GCP",
    r"\bbig(\s)?query\b" : "Big Query",
    #cms(Content Management System)
    r"\bdrupal\b" : "Drupal",
    #erp
    r"\bnetsuite\b" : "NetSuite",
    r"\bsap\b" : "SAP",
    r"\sbw(4hana)?\s" : "BW4HANA",
    #crm (Customer Relationship Management)
    r"\bsalesforce\b" : "salesforce",
    #control de versiones
    r"\bgit\b" : "Git",
    r"\bbit(\s)?bucket\b" : "BitBucket",
    r"\bgithub\b" : "Github",
    #un motor de búsqueda y análisis para todo tipo de datos
    r"elastic(\s)?search" : "Elastic Search",
    #diseno
    r"\bfigma\b" : "Figma",
    r"\bautocad\b" : "AutoCAD",
    r"\bui\b" : "UI",
    r"\bxd\b" : "XD",
    r"\bzeplin\b" : "Zeplin",
    r"\binvision\b" : "Invision",
    #visualizacion
    r"\btableau\b" : "Tableau",
    r"\bpower(\s)?bi\b" : "Power BI", 
    r"\bqlik(\s)?sense\b" : "Qlik Sense",
    r"\bqlik(\s)?view\b" : "QlikView", 
    #api and services
    r"\brest\sapi\b" : "REST API",
    r"\bgraphql\b" : "graphql",
    r"\bweb(\s)api?\b" : "Web API",
    r"\bmicroservices\b" :"Microservices",
    r"\bsoa\b" :"SOA",
    #
    r"\bfacebook(\s)ads\b" : "Facebook Ads",
    r"\bgoogle(\s)analytics\b" : "Google Analytics",
    r"\bseo\b" : "SEO",
    #
    r"\bappium\b" :"Appium",
    r"\bcucumber\b" : "Cucumber",  
    r"\btest\scases\b" : "Test cases",
    r"\bautomation(\s)testing\b" : "Automation Testing",
    r"\bfuncional\stesting\b" : "Funcional Testing",
    #
    r"\bsaltstack\b" : "Saltstack",
    r"\bterraform\b" : "Terraform",
    r"\bpuppet\b" : "Puppet",
    r"\bchef\b" : "Chef",
    #virtualizacion
    r"\bkvm\b" : "KVM",
    r"\bxen\b" : "Xen",
    r"\bvmware\b" : "VMWare",
    r"\bvirtualbox\b" : "VirtualBox",
    #
    r"\bcisco\b" : "Cisco",
    r"\bhp\sopen\sview\b" : "HP Open View",
    r"\bnetgear\b" : "Netgear",
    r"\bap\sfortinet\b" : "AP Fortinet",
    #otros
    r"\bscraping\b" : "Scraping",
    r"\bdomo\b" : "Domo" ,
    r"\buml\b" : "UML" 
}

In [ ]:
keys_degree = diccionario_academic_degree.keys()
keys_soft_skill = diccionario_soft_skill.keys()
keys_hardSkill = diccionario_HardSkill.keys()
keys_idiomas = diccionario_languages.keys()

def ObtenerGradoEducativo(df, columna_descripcion):
  resultados_degree = []
  for i in range(len(df)):
    resultado_fila_degree = ""
    for key in keys_degree:  
      pattern = re.compile(key, re.IGNORECASE)
      #print( type(df[columna_descripcion][i]) )
      try:
        if pattern.search(df[columna_descripcion][i]):
          resultado_fila_degree = resultado_fila_degree + diccionario_academic_degree[key] + ", " 
      except:
        pass
    #Resultados grado academico
    resultado_fila_degree = resultado_fila_degree[0:-2]
    resultados_degree.append(resultado_fila_degree)  

  return resultados_degree

def ObtenerIdiomas(df, columna_descripcion):
  resultados_idiomas = []
  for i in range(len(df)):
    resultado_fila_idioma = ""
    for key in keys_idiomas:  
      pattern = re.compile(key, re.IGNORECASE)
      #print( type(df[columna_descripcion][i]) )
      try:
        if pattern.search(df[columna_descripcion][i]):
          resultado_fila_idioma = resultado_fila_idioma + diccionario_languages[key] + ", " 
      except:
        pass
    #Resultados grado academico
    resultado_fila_idioma = resultado_fila_idioma[0:-2]
    resultados_idiomas.append(resultado_fila_idioma)  

  return resultados_idiomas

def ObtenerYears(df, columna_descripcion):
  resultados_years= []
  for i in range(len(df)):
    resultado_fila_years_experience = ""
    try:
      if patron_year.search(df[columna_descripcion][i]):
        match_year = patron_year.findall(df[columna_descripcion][i])
        for m in match_year:
          resultado_fila_years_experience = resultado_fila_years_experience + m[0] + ", "
    except:
      pass
    resultado_fila_years_experience = resultado_fila_years_experience[0:-2]
    resultados_years.append(resultado_fila_years_experience) 
  return resultados_years

def ObtenerSoftSkill(df, columna_descripcion):
  resultados_soft_skill = []
  for i in range(len(df)):
    resultado_fila_soft_skill = ""
    for key in keys_soft_skill:
      pattern = re.compile(key, re.IGNORECASE)
      try:
        if pattern.search(df[columna_descripcion][i]):
          resultado_fila_soft_skill = resultado_fila_soft_skill + diccionario_soft_skill[key] + ", "
      except:
        pass
    #Resultados de fila Soft skill
    resultado_fila_soft_skill = resultado_fila_soft_skill[0:-2]
    resultados_soft_skill.append(resultado_fila_soft_skill)  

  return resultados_soft_skill

def ObtenerSkill(df, columna_descripcion):
  resultados_skill= []
  for i in range(len(df)):
    resultado_fila_skill = ""
    for key in keys_hardSkill:
      pattern = re.compile(key, re.IGNORECASE)
      try:
        if pattern.search(df[columna_descripcion][i]):
              resultado_fila_skill = resultado_fila_skill + diccionario_HardSkill[key] + ", "
      except: 
        pass
    #Resultados skill Front End
    resultado_fila_skill = resultado_fila_skill[0:-2]
    resultados_skill.append(resultado_fila_skill)  

  return resultados_skill

In [ ]:
#Patron de remote
busqueda_remote = r"remot(o|e)"
patron_remote= re.compile(busqueda_remote, re.IGNORECASE)

def isRemote(df, columna_info):
  resultados_remote= []
  for i in range(len(df)):
    resultado_fila_remote = ""
    try:
      if patron_remote.search(df[columna_info][i]):
        resultado_fila_remote = "Remote"
      else:
        resultado_fila_remote = None
    except:
      pass
    resultados_remote.append(resultado_fila_remote) 
  return resultados_remote

In [ ]:
estados_Mexico ={
    r'aguascalientes' : "Aguascalientes",  r'baja\scalifornia' : 'Baja California', 
    r'baja\scalifornia\ssur' : 'Baja California Sur',  r'campeche' : 'Campeche', 
    r'chiapas':'Chiapas', r'chihuahua' :'Chihuahua', 
    r'ciudad\sde\sm(é|e)xico' : 'Ciudad de Mexico', 
    r'coahuila' : 'Coahuila', r'colima' : 'Colima', 
    r'durango' : 'durango', r"estado\sde\sm(é|e)xico" : "Estado de mexico",
    r'guanajuato' : 'Guanajuato', r'guerrero' : 'Guerrero', 
    r'hidalgo' : 'Hidalgo', r'jalisco' : 'Jalisco', 
    r'michoac(á|a)n' : 'Michoacan', r'morelos' : 'Morelos', 
    r'nayarit' : 'Nayarit', r'nuevo\sLe(ó|o)n' : "Nuevo Leon", 
    r'oaxaca' : "Oaxaca", r'puebla' : "Puebla", r'quer(é|e)taro' : 'Queretaro', 
    r'quintana\sroo' : "Quintana Roo", r'san\sluis\spotos(í|i)': 'San Luis Potosí', 
    r'sinaloa' : 'Sinaloa', r'sonora' : 'Sonora', 
    r'tabasco' : 'Tabasco', r'tamaulipas' : 'Tamaulipas', 
    r'tlaxcala' : 'Tlaxcala', r'veracruz' : 'Veracruz', 
    r'yucat(á|a)n' : 'Yucatan', r'zacatecas' : 'Zacatecas', 
    r"(remot(o|e)|trabaj(a|o)\sdesde\scasa)" : "Remoto"}

keys_estados = estados_Mexico.keys()
def ObtenerEstado(df, columna):
    estados = df[columna].str.split(",").str.get(1)
    for i in range (len(estados)):
        if(type(estados[i]) == float):
            for key in keys_estados:
                pattern = re.compile(key, re.IGNORECASE)
                try:
                    if pattern.search(df[columna][i]):
                        estados[i] = estados_Mexico[key]
                        continue
                except: 
                    continue
    return estados


In [ ]:
def ExtraerInfo_Empleos_TI():
    Title = ""
    Location = ""
    Company = ""
    Salary = ""
    Tipo_Puesto = ""
    English = ""
    Description = ""
    Logo = ""
    #Obtener el nombre del puesto
    try:
        title_driver = driver.find_element_by_xpath('//h1[@class="section-title"]')
        Title = title_driver.text
    except:
        Title = None
    
    #Locacion, tipo de puesto, english and salary 
    try:
        datos_driver = driver.find_elements_by_xpath('//ul[@class="field-set-vacancy"]//p[@class="gray-subarea-title"]')
        try:
            Salary = datos_driver[0].text
        except:
            Salary = None

        try:
            Location = datos_driver[1].text
        except:
            Location = None

        try:
            Tipo_Puesto = datos_driver[2].text
        except:
            Tipo_Puesto = None

        try:
            English = datos_driver[3].text
        except:
            English = None    
    except :
        Salary = None
        Location = None
        Tipo_Puesto = None
        English = None  
    #Nombre de la compania
    try:
        company_driver = driver.find_element_by_id('searchable-enterprise')
        Company = company_driver.text
    except :
        Company = None
    #Descripcion
    try:
        description_driver = driver.find_element_by_id('vacancy-description')
        Description = description_driver.text
    except:
        Description = None
    resultados = {"Title" : Title, "Location" : Location, "Company" : Company, "Description" : Description,  "Salary" : Salary, "Tipo_Puesto" : Tipo_Puesto, "English" : English}

    return resultados

In [ ]:
persona = "Cecy"
if(persona == "Mario"):
  #Mario
  terminos = ["Java", ".Net/C#", "JavaScript", "PHP", "AngularJS", "ReactJS", "Python", "Diseño de software", "Android"]
elif(persona == "Ian"):
  #Ian
  terminos = ["IOS", "PLSQL", "Ruby", "Cobol", "C++", "Testing/QA", "SAP", "Analista de sistemas", "Administracion de proyectos"]
elif(persona == "Jezrrel"):
  #Jezrrel
  terminos = ["Redes", "Auditoria de seguridad", "Ciberseguridad", "Pentesting", "Sysadmin", "Devops", "Administrador de base de datos", "Business intelligence", "Data Warehouse"]
elif(persona == "Cecy"):
  #Cecy
  terminos = ["Big data", "Data Science", "Sales force", "UI UX", "MKT digital", "SEO", "SEM", "Ventas TI", "Reclutamiento TI"]

In [ ]:
terminos

['Big data',
 'Data Science',
 'Sales force',
 'UI UX',
 'MKT digital',
 'SEO',
 'SEM',
 'Ventas TI',
 'Reclutamiento TI']

In [ ]:
#Ingresar el termino que se usara en el cuadro de busqueda

#test
#terminos = ["SEO"]

#Creo el dataframe
df_Empleos_TI = pd.DataFrame(columns = ["Title", "Location", "Company" ,"Description", "Salary", "Tipo_Puesto", "English"])
driver_path = "chromedriver"
driver = webdriver.Chrome(executable_path=driver_path,options=chrome_options)

#Recorrer la lista
for termino in terminos:
  #pagina principal
  driver.get("https://empleosti.com.mx/empleos?k=&l=")
  #cuadro de busqueda
  element = driver.find_element(By.XPATH,'//input[@id="input-k"]')
  #ingresar el nombre de la company
  element.send_keys(termino)
  #enter
  element.send_keys(Keys.ENTER)
  #espera
  WebDriverWait(driver, 10)
  try:
    next = driver.find_element(By.PARTIAL_LINK_TEXT, 'chevron_right')  
    #driver.get(next.get_attribute("href"))
    url_base = next.get_attribute("href")
    url_base = url_base[:-1]
    
  except:
    url_base = driver.current_url
  print(driver.current_url)
  #Revisa cuantas paginas hay
  paginas = driver.find_elements(By.XPATH,'//a[@class="page-link"]')
  try:
    ultima_pagina = int(paginas[-1].text)
  except:
    pass
  empleos_pagina = driver.find_elements(By.XPATH,'//div[@class="noContainerUp container"]//ul[@class="job-eti"]//li')  
  #Obtener datos de la primera pagina
  lista_empleos = driver.find_elements(By.XPATH,'//div[@class="noContainerUp container"]//ul[@class="job-eti"]//a')       
  lista =[]

  for job in lista_empleos:
      lista.append(job.get_attribute("href"))

  for link in lista:
      #print(link)
      driver.get(link)
      sleep(2)
      df_Empleos_TI = df_Empleos_TI.append(ExtraerInfo_Empleos_TI(), ignore_index=True)

  if(len(paginas)>=2):
    for i in range (2,ultima_pagina + 1):
      url_completo = url_base + str(i)
      print(url_completo)
      driver.get(url_completo)
      #datos de demas paginas
      lista_empleos = driver.find_elements(By.XPATH,'//div[@class="noContainerUp container"]//ul[@class="job-eti"]//a')       
      lista =[]
      
      for job in lista_empleos:
          lista.append(job.get_attribute("href"))

      for link in lista:
          #print(link)
          driver.get(link)
          sleep(2)
          df_Empleos_TI = df_Empleos_TI.append(ExtraerInfo_Empleos_TI(), ignore_index=True)



In [ ]:
df_Empleos_TI

,Title,Location,Company,Description,Salary,Tipo_Puesto,English,Academy_grade,Years_Experience,SoftSkill,HardSkill,Remoto,Estado,Accessed_Date
0,Data Architecht,Remoto - Solo México,Pypol Soluciones,Nos encontramos en búsqueda de un Data Archite...,"$ 50,000 a 60,000",Empleado de tiempo completo,Nivel Intermedio,,,,,Remote,Remoto,2021-10-26
1,Arquitecto Google Cloud Platform,"Miguel Hidalgo, Ciudad de México",Pypol Soluciones,Estamos buscando un Arquitecto Google Cloud Pl...,"$ 45,000 a 50,000",Empleado de tiempo completo,Nivel Intermedio,,1 año,,,None,Ciudad de México,2021-10-26
2,Data Analyst,"Miguel Hidalgo, Ciudad de México",Pypol Soluciones,Nos encontramos en búsqueda de un Data Analyst...,"$ 35,000 a 40,000",Empleado de tiempo completo,Nivel Intermedio,Licenciatura,2-3 años,,SQL,None,Ciudad de México,2021-10-26
3,Consultor SSAS,"Miguel Hidalgo, Ciudad de México",Pypol Soluciones,Nos encontramos en búsqueda de un Consultor SS...,"$ 35,000 a 40,000",Empleado de tiempo completo,Nivel Intermedio,,3 años,,SQL,None,Ciudad de México,2021-10-26
4,Desarrollador Big Data,"Miguel Hidalgo, Ciudad de México",Pypol Soluciones,Nos encontramos en búsqueda de Desarrolladores...,"$ 45,000 a 50,000",Empleado de tiempo completo,Nivel Intermedio,Licenciatura,,,"Java, Python, Oracle, NoSQL, Cassandra",None,Ciudad de México,2021-10-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,Project Manager (Inglés Avanzado),"Venustiano Carranza, Ciudad de México",CAPITAL EMPRESARIAL HORIZONTE SA DE CV,Capital Empresarial Horizonte Empresa Mexicana...,"$ 15,000 a 20,000",Empleado de tiempo completo,Nivel Avanzado,Licenciatura,3 años,"Capacidad Analitica, Organizado, Trabajo en eq...",,None,Ciudad de México,2021-10-26
1114,63-1759 Jefe de Turno,"Miguel Hidalgo, Ciudad de México",Involve,¡Prepárate para formar parte de un gran equipo...,No especificado,Empleado de tiempo completo,Nivel Básico,,,Creatividad,,None,Ciudad de México,2021-10-26
1115,Arquitecto SCCM,"Benito Juárez, Ciudad de México",CAPITAL EMPRESARIAL HORIZONTE SA DE CV,Capital Empresarial Horizonte Empresa Mexicana...,"$ 50,000 a 60,000",Empleado de tiempo completo,Nivel Avanzado,Licenciatura,3 años,,,None,Ciudad de México,2021-10-26
1116,DevOps (Inglés Conversacional),"Benito Juárez, Ciudad de México",CAPITAL EMPRESARIAL HORIZONTE SA DE CV,Capital Empresarial Horizonte Empresa Mexicana...,"$ 45,000 a 50,000",Empleado de tiempo completo,Nivel Avanzado,Licenciatura,3 años,,"Git, Docker, Kubernetes, Devops, Azure",None,Ciudad de México,2021-10-26


In [ ]:
"""Obtener grado educativo, años de experiencia, english, soft skills, 
skills front y back end a traves de expresiones regulares """
df_Empleos_TI["Academy_grade"] = ObtenerGradoEducativo(df_Empleos_TI, "Description") 
df_Empleos_TI["Years_Experience"] = ObtenerYears(df_Empleos_TI, "Description")
df_Empleos_TI["SoftSkill"] = ObtenerSoftSkill(df_Empleos_TI, "Description") 
df_Empleos_TI["HardSkill"] = ObtenerSkill(df_Empleos_TI, "Description") 


In [ ]:
#Columnas que aparecen en stackoverflow, pero aqui no
df_Empleos_TI["Remoto"] = isRemote(df_Empleos_TI, "Location")
#df_Empleos_TI["Visa_sponsor"] = None
#df_Empleos_TI["Paid_relocation"] = None
#Columnas que aparecen en indeed, pero aqui no
#df_Empleos_TI["ID"] = None
df_Empleos_TI["Estado"] = ObtenerEstado(df_Empleos_TI, "Location")
#df_Empleos_TI["Synopsis"] = None
#df_Empleos_TI["Post_Date"] = None
df_Empleos_TI["Accessed_Date"] = date.today()
#Columnas que aparecen en Glassdoor, pero aqui no
#df_Empleos_TI["Min_Salary"] = None
#df_Empleos_TI["Max_Salary"] = None
#df_Empleos_TI["Pay_Period"] = None
#df_Empleos_TI["Company_Overview"] = None

In [ ]:
# Save file
df_Empleos_TI.to_csv('/content/drive/MyDrive/Colab Notebooks/Empleos_TI/data/Empleos_TI_{}_{}.csv'.format(persona, date.today().strftime('%Y%m%d')), index=False)